In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import torch
import os
import sys
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import accuracy_score
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pickle
import numpy as np

import sys
sys.path.append("../../../src/")
sys.path.append("../..")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid, \
     get_gt_pred_specific_class, get_differ
from aux.visualization import visualize_features_map
from utils.function import recreate_image                                                                                                                                       

## Hyperparameter

In [3]:
resume = "037-0"
# model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
# generated_dir = "/home/lincolnzjx/Desktop/saved/generated/"
model_dir = "/data15/jiaxin/myCode/Interpretation/saved/models"
generated_dir = "/data15/jiaxin/myCode/saved/generated/"
root = "/data15/Public/Datasets/"
backbone = "vgg16"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load modal from dict

In [4]:
# model
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)

# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt) 

# Class & data
batch_size = 256
num_workers = 4
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]

> Use original fc
Resume from model from exp: 037 at epoch 0


## Load Swap, RGB->BGR

In [5]:
bgr_class_y_true_y_pred_acc = {}
for class_index in range(1000):
    # Load data
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root=root, 
                                 is_train=True, transform=train_transform)
    len_specific_category = len(trainset.targets_imgs_dict[class_index])
    print(len_specific_category)
    trainset.set_data([class_index], len_specific_category)
    trainloader = torch.utils.data.DataLoader(trainset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=num_workers,
                                              pin_memory=True)
    y_true, y_pred, acc = get_gt_pred_specific_class(net, 
                                                     trainloader,
                                                     order=[2,1,0],
                                                     device=device)
    print("class_index:{}, Acc1 {}".format(class_index, acc))
    bgr_class_y_true_y_pred_acc[class_index] = [y_true, y_pred, acc]

1300
Len of new dataset is :1300
class_index:0, Acc1 0.48846153846153845
1300
Len of new dataset is :1300
class_index:1, Acc1 0.2523076923076923
1300
Len of new dataset is :1300
class_index:2, Acc1 0.3861538461538462
1300
Len of new dataset is :1300
class_index:3, Acc1 0.47846153846153844
1300
Len of new dataset is :1300
class_index:4, Acc1 0.6676923076923077
1300
Len of new dataset is :1300
class_index:5, Acc1 0.5592307692307692
1300
Len of new dataset is :1300
class_index:6, Acc1 0.6853846153846154
1300
Len of new dataset is :1300
class_index:7, Acc1 0.5146153846153846
1300
Len of new dataset is :1300
class_index:8, Acc1 0.47692307692307695
1300
Len of new dataset is :1300
class_index:9, Acc1 0.8707692307692307
1300
Len of new dataset is :1300
class_index:10, Acc1 0.2923076923076923
1300
Len of new dataset is :1300
class_index:11, Acc1 0.15384615384615385
1300
Len of new dataset is :1300
class_index:12, Acc1 0.29307692307692307
1300
Len of new dataset is :1300
class_index:13, Acc1 0.

1300
Len of new dataset is :1300
class_index:113, Acc1 0.6484615384615384
1300
Len of new dataset is :1300
class_index:114, Acc1 0.5507692307692308
1300
Len of new dataset is :1300
class_index:115, Acc1 0.6007692307692307
1300
Len of new dataset is :1300
class_index:116, Acc1 0.7207692307692307
1300
Len of new dataset is :1300
class_index:117, Acc1 0.8553846153846154
1300
Len of new dataset is :1300
class_index:118, Acc1 0.25461538461538463
1300
Len of new dataset is :1300
class_index:119, Acc1 0.48384615384615387
1300
Len of new dataset is :1300
class_index:120, Acc1 0.6730769230769231
1300
Len of new dataset is :1300
class_index:121, Acc1 0.2653846153846154
1300
Len of new dataset is :1300
class_index:122, Acc1 0.41923076923076924
1300
Len of new dataset is :1300
class_index:123, Acc1 0.7823076923076923
1300
Len of new dataset is :1300
class_index:124, Acc1 0.4238461538461539
1300
Len of new dataset is :1300
class_index:125, Acc1 0.5584615384615385
1300
Len of new dataset is :1300
cl

1300
Len of new dataset is :1300
class_index:225, Acc1 0.05615384615384615
1300
Len of new dataset is :1300
class_index:226, Acc1 0.4076923076923077
1300
Len of new dataset is :1300
class_index:227, Acc1 0.5907692307692308
1300
Len of new dataset is :1300
class_index:228, Acc1 0.8623076923076923
1300
Len of new dataset is :1300
class_index:229, Acc1 0.693076923076923
1300
Len of new dataset is :1300
class_index:230, Acc1 0.44076923076923075
1300
Len of new dataset is :1300
class_index:231, Acc1 0.3515384615384615
1300
Len of new dataset is :1300
class_index:232, Acc1 0.6238461538461538
1300
Len of new dataset is :1300
class_index:233, Acc1 0.5169230769230769
1300
Len of new dataset is :1300
class_index:234, Acc1 0.013846153846153847
1300
Len of new dataset is :1300
class_index:235, Acc1 0.1176923076923077
1300
Len of new dataset is :1300
class_index:236, Acc1 0.20461538461538462
1300
Len of new dataset is :1300
class_index:237, Acc1 0.2592307692307692
1300
Len of new dataset is :1300
c

class_index:337, Acc1 0.6107692307692307
1300
Len of new dataset is :1300
class_index:338, Acc1 0.7169230769230769
1300
Len of new dataset is :1300
class_index:339, Acc1 0.3607692307692308
1300
Len of new dataset is :1300
class_index:340, Acc1 0.95
1300
Len of new dataset is :1300
class_index:341, Acc1 0.4176923076923077
1300
Len of new dataset is :1300
class_index:342, Acc1 0.6146153846153846
1300
Len of new dataset is :1300
class_index:343, Acc1 0.75
1300
Len of new dataset is :1300
class_index:344, Acc1 0.6861538461538461
1300
Len of new dataset is :1300
class_index:345, Acc1 0.5153846153846153
1300
Len of new dataset is :1300
class_index:346, Acc1 0.5869230769230769
1300
Len of new dataset is :1300
class_index:347, Acc1 0.7176923076923077
1300
Len of new dataset is :1300
class_index:348, Acc1 0.5769230769230769
1300
Len of new dataset is :1300
class_index:349, Acc1 0.7376923076923076
1300
Len of new dataset is :1300
class_index:350, Acc1 0.7076923076923077
1300
Len of new dataset i

1300
Len of new dataset is :1300
class_index:450, Acc1 0.8292307692307692
1300
Len of new dataset is :1300
class_index:451, Acc1 0.7723076923076924
1300
Len of new dataset is :1300
class_index:452, Acc1 0.713076923076923
1300
Len of new dataset is :1300
class_index:453, Acc1 0.6715384615384615
1300
Len of new dataset is :1300
class_index:454, Acc1 0.6915384615384615
1300
Len of new dataset is :1300
class_index:455, Acc1 0.7492307692307693
1300
Len of new dataset is :1300
class_index:456, Acc1 0.5915384615384616
1300
Len of new dataset is :1300
class_index:457, Acc1 0.5315384615384615
1300
Len of new dataset is :1300
class_index:458, Acc1 0.8530769230769231
1300
Len of new dataset is :1300
class_index:459, Acc1 0.5646153846153846
1300
Len of new dataset is :1300
class_index:460, Acc1 0.546923076923077
1300
Len of new dataset is :1300
class_index:461, Acc1 0.5261538461538462
1300
Len of new dataset is :1300
class_index:462, Acc1 0.6707692307692308
1300
Len of new dataset is :1300
class_i

1300
Len of new dataset is :1300
class_index:562, Acc1 0.7984615384615384
1300
Len of new dataset is :1300
class_index:563, Acc1 0.66
1300
Len of new dataset is :1300
class_index:564, Acc1 0.8853846153846154
1300
Len of new dataset is :1300
class_index:565, Acc1 0.86
1300
Len of new dataset is :1300
class_index:566, Acc1 0.75
1222
Len of new dataset is :1222
class_index:567, Acc1 0.35515548281505727
1300
Len of new dataset is :1300
class_index:568, Acc1 0.5930769230769231
1300
Len of new dataset is :1300
class_index:569, Acc1 0.8369230769230769
1300
Len of new dataset is :1300
class_index:570, Acc1 0.7069230769230769
1300
Len of new dataset is :1300
class_index:571, Acc1 0.7561538461538462
1300
Len of new dataset is :1300
class_index:572, Acc1 0.7992307692307692
1300
Len of new dataset is :1300
class_index:573, Acc1 0.806923076923077
1300
Len of new dataset is :1300
class_index:574, Acc1 0.8438461538461538
1300
Len of new dataset is :1300
class_index:575, Acc1 0.8576923076923076
1300
L

1300
Len of new dataset is :1300
class_index:674, Acc1 0.65
1159
Len of new dataset is :1159
class_index:675, Acc1 0.6859361518550474
1133
Len of new dataset is :1133
class_index:676, Acc1 0.6443071491615181
1300
Len of new dataset is :1300
class_index:677, Acc1 0.6738461538461539
1180
Len of new dataset is :1180
class_index:678, Acc1 0.538135593220339
1300
Len of new dataset is :1300
class_index:679, Acc1 0.8223076923076923
1300
Len of new dataset is :1300
class_index:680, Acc1 0.6176923076923077
1300
Len of new dataset is :1300
class_index:681, Acc1 0.5069230769230769
1300
Len of new dataset is :1300
class_index:682, Acc1 0.8938461538461538
1300
Len of new dataset is :1300
class_index:683, Acc1 0.5538461538461539
1300
Len of new dataset is :1300
class_index:684, Acc1 0.7092307692307692
1300
Len of new dataset is :1300
class_index:685, Acc1 0.9207692307692308
1120
Len of new dataset is :1120
class_index:686, Acc1 0.7017857142857142
1300
Len of new dataset is :1300
class_index:687, Acc

/data1/jiaxin/anaconda3/envs/Py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


class_index:782, Acc1 0.3820047355958958
1300
Len of new dataset is :1300
class_index:783, Acc1 0.6607692307692308
1300
Len of new dataset is :1300
class_index:784, Acc1 0.40615384615384614
1300
Len of new dataset is :1300
class_index:785, Acc1 0.6169230769230769
1300
Len of new dataset is :1300
class_index:786, Acc1 0.8115384615384615
1300
Len of new dataset is :1300
class_index:787, Acc1 0.6738461538461539
1300
Len of new dataset is :1300
class_index:788, Acc1 0.693076923076923
1249
Len of new dataset is :1249
class_index:789, Acc1 0.6236989591673339
1300
Len of new dataset is :1300
class_index:790, Acc1 0.7092307692307692
1300
Len of new dataset is :1300
class_index:791, Acc1 0.7584615384615384
1300
Len of new dataset is :1300
class_index:792, Acc1 0.5553846153846154
1300
Len of new dataset is :1300
class_index:793, Acc1 0.7353846153846154
1300
Len of new dataset is :1300
class_index:794, Acc1 0.8
1300
Len of new dataset is :1300
class_index:795, Acc1 0.7115384615384616
1300
Len of 

1300
Len of new dataset is :1300
class_index:895, Acc1 0.7392307692307692
1300
Len of new dataset is :1300
class_index:896, Acc1 0.6984615384615385
1300
Len of new dataset is :1300
class_index:897, Acc1 0.8507692307692307
1300
Len of new dataset is :1300
class_index:898, Acc1 0.4453846153846154
1300
Len of new dataset is :1300
class_index:899, Acc1 0.31846153846153846
1300
Len of new dataset is :1300
class_index:900, Acc1 0.9069230769230769
1055
Len of new dataset is :1055
class_index:901, Acc1 0.7364928909952607
1300
Len of new dataset is :1300
class_index:902, Acc1 0.4753846153846154
1300
Len of new dataset is :1300
class_index:903, Acc1 0.6638461538461539
1300
Len of new dataset is :1300
class_index:904, Acc1 0.6638461538461539
1300
Len of new dataset is :1300
class_index:905, Acc1 0.696923076923077
1213
Len of new dataset is :1213
class_index:906, Acc1 0.5581203627370157
1300
Len of new dataset is :1300
class_index:907, Acc1 0.566923076923077
1300
Len of new dataset is :1300
class_

In [11]:
bgr_gt_pred_acc = "./bgr_gt_pred_acc.pickle"
with open(bgr_gt_pred_acc, "wb") as handle:
    pickle.dump(bgr_class_y_true_y_pred_acc, handle)
print(np.mean([acc for _, _, acc in bgr_class_y_true_y_pred_acc.values()]))

0.6085282342545499


## Load Swap, RGB->RBG

In [12]:
rbg_class_y_true_y_pred_acc = {}
for class_index in range(1000):
    # Load data
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root=root, 
                                 is_train=True, transform=train_transform)
    len_specific_category = len(trainset.targets_imgs_dict[class_index])
    print(len_specific_category)
    trainset.set_data([class_index], len_specific_category)
    trainloader = torch.utils.data.DataLoader(trainset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=num_workers,
                                              pin_memory=True)
    y_true, y_pred, acc = get_gt_pred_specific_class(net, 
                                                     trainloader,
                                                     order=[0,2,1],
                                                     device=device)
    print("class_index:{}, Acc1 {}".format(class_index, acc))
    rbg_class_y_true_y_pred_acc[class_index] = [y_true, y_pred, acc]

1300
Len of new dataset is :1300
class_index:0, Acc1 0.4653846153846154
1300
Len of new dataset is :1300
class_index:1, Acc1 0.7269230769230769
1300
Len of new dataset is :1300
class_index:2, Acc1 0.7276923076923076
1300
Len of new dataset is :1300
class_index:3, Acc1 0.7238461538461538
1300
Len of new dataset is :1300
class_index:4, Acc1 0.7146153846153847
1300
Len of new dataset is :1300
class_index:5, Acc1 0.62
1300
Len of new dataset is :1300
class_index:6, Acc1 0.7161538461538461
1300
Len of new dataset is :1300
class_index:7, Acc1 0.73
1300
Len of new dataset is :1300
class_index:8, Acc1 0.4846153846153846
1300
Len of new dataset is :1300
class_index:9, Acc1 0.8223076923076923
1300
Len of new dataset is :1300
class_index:10, Acc1 0.19230769230769232
1300
Len of new dataset is :1300
class_index:11, Acc1 0.04384615384615385
1300
Len of new dataset is :1300
class_index:12, Acc1 0.8323076923076923
1300
Len of new dataset is :1300
class_index:13, Acc1 0.6615384615384615
1300
Len of ne

class_index:113, Acc1 0.5638461538461539
1300
Len of new dataset is :1300
class_index:114, Acc1 0.4053846153846154
1300
Len of new dataset is :1300
class_index:115, Acc1 0.5953846153846154
1300
Len of new dataset is :1300
class_index:116, Acc1 0.6461538461538462
1300
Len of new dataset is :1300
class_index:117, Acc1 0.8530769230769231
1300
Len of new dataset is :1300
class_index:118, Acc1 0.21307692307692308
1300
Len of new dataset is :1300
class_index:119, Acc1 0.42538461538461536
1300
Len of new dataset is :1300
class_index:120, Acc1 0.5676923076923077
1300
Len of new dataset is :1300
class_index:121, Acc1 0.3869230769230769
1300
Len of new dataset is :1300
class_index:122, Acc1 0.6376923076923077
1300
Len of new dataset is :1300
class_index:123, Acc1 0.6646153846153846
1300
Len of new dataset is :1300
class_index:124, Acc1 0.4946153846153846
1300
Len of new dataset is :1300
class_index:125, Acc1 0.553076923076923
1300
Len of new dataset is :1300
class_index:126, Acc1 0.5553846153846

class_index:225, Acc1 0.27
1300
Len of new dataset is :1300
class_index:226, Acc1 0.4346153846153846
1300
Len of new dataset is :1300
class_index:227, Acc1 0.52
1300
Len of new dataset is :1300
class_index:228, Acc1 0.8538461538461538
1300
Len of new dataset is :1300
class_index:229, Acc1 0.7592307692307693
1300
Len of new dataset is :1300
class_index:230, Acc1 0.41923076923076924
1300
Len of new dataset is :1300
class_index:231, Acc1 0.2853846153846154
1300
Len of new dataset is :1300
class_index:232, Acc1 0.6892307692307692
1300
Len of new dataset is :1300
class_index:233, Acc1 0.5876923076923077
1300
Len of new dataset is :1300
class_index:234, Acc1 0.17461538461538462
1300
Len of new dataset is :1300
class_index:235, Acc1 0.40076923076923077
1300
Len of new dataset is :1300
class_index:236, Acc1 0.41307692307692306
1300
Len of new dataset is :1300
class_index:237, Acc1 0.4684615384615385
1300
Len of new dataset is :1300
class_index:238, Acc1 0.3853846153846154
1300
Len of new datas

1300
Len of new dataset is :1300
class_index:339, Acc1 0.7930769230769231
1300
Len of new dataset is :1300
class_index:340, Acc1 0.92
1300
Len of new dataset is :1300
class_index:341, Acc1 0.5176923076923077
1300
Len of new dataset is :1300
class_index:342, Acc1 0.4576923076923077
1300
Len of new dataset is :1300
class_index:343, Acc1 0.6353846153846154
1300
Len of new dataset is :1300
class_index:344, Acc1 0.7961538461538461
1300
Len of new dataset is :1300
class_index:345, Acc1 0.46153846153846156
1300
Len of new dataset is :1300
class_index:346, Acc1 0.6007692307692307
1300
Len of new dataset is :1300
class_index:347, Acc1 0.6707692307692308
1300
Len of new dataset is :1300
class_index:348, Acc1 0.4846153846153846
1300
Len of new dataset is :1300
class_index:349, Acc1 0.6676923076923077
1300
Len of new dataset is :1300
class_index:350, Acc1 0.7146153846153847
1300
Len of new dataset is :1300
class_index:351, Acc1 0.6423076923076924
1300
Len of new dataset is :1300
class_index:352, A

1300
Len of new dataset is :1300
class_index:451, Acc1 0.7338461538461538
1300
Len of new dataset is :1300
class_index:452, Acc1 0.7376923076923076
1300
Len of new dataset is :1300
class_index:453, Acc1 0.76
1300
Len of new dataset is :1300
class_index:454, Acc1 0.5707692307692308
1300
Len of new dataset is :1300
class_index:455, Acc1 0.7369230769230769
1300
Len of new dataset is :1300
class_index:456, Acc1 0.6130769230769231
1300
Len of new dataset is :1300
class_index:457, Acc1 0.5515384615384615
1300
Len of new dataset is :1300
class_index:458, Acc1 0.7653846153846153
1300
Len of new dataset is :1300
class_index:459, Acc1 0.5930769230769231
1300
Len of new dataset is :1300
class_index:460, Acc1 0.3861538461538462
1300
Len of new dataset is :1300
class_index:461, Acc1 0.42846153846153845
1300
Len of new dataset is :1300
class_index:462, Acc1 0.7146153846153847
1300
Len of new dataset is :1300
class_index:463, Acc1 0.6046153846153847
1300
Len of new dataset is :1300
class_index:464, A

1300
Len of new dataset is :1300
class_index:566, Acc1 0.6923076923076923
1222
Len of new dataset is :1222
class_index:567, Acc1 0.31342062193126025
1300
Len of new dataset is :1300
class_index:568, Acc1 0.5115384615384615
1300
Len of new dataset is :1300
class_index:569, Acc1 0.8761538461538462
1300
Len of new dataset is :1300
class_index:570, Acc1 0.7223076923076923
1300
Len of new dataset is :1300
class_index:571, Acc1 0.7807692307692308
1300
Len of new dataset is :1300
class_index:572, Acc1 0.76
1300
Len of new dataset is :1300
class_index:573, Acc1 0.8
1300
Len of new dataset is :1300
class_index:574, Acc1 0.8084615384615385
1300
Len of new dataset is :1300
class_index:575, Acc1 0.8646153846153846
1300
Len of new dataset is :1300
class_index:576, Acc1 0.9415384615384615
1282
Len of new dataset is :1282
class_index:577, Acc1 0.6677067082683308
1300
Len of new dataset is :1300
class_index:578, Acc1 0.5523076923076923
1300
Len of new dataset is :1300
class_index:579, Acc1 0.706923076

1300
Len of new dataset is :1300
class_index:679, Acc1 0.83
1300
Len of new dataset is :1300
class_index:680, Acc1 0.7192307692307692
1300
Len of new dataset is :1300
class_index:681, Acc1 0.37846153846153846
1300
Len of new dataset is :1300
class_index:682, Acc1 0.8615384615384616
1300
Len of new dataset is :1300
class_index:683, Acc1 0.5661538461538461
1300
Len of new dataset is :1300
class_index:684, Acc1 0.7246153846153847
1300
Len of new dataset is :1300
class_index:685, Acc1 0.9515384615384616
1120
Len of new dataset is :1120
class_index:686, Acc1 0.6848214285714286
1300
Len of new dataset is :1300
class_index:687, Acc1 0.74
1300
Len of new dataset is :1300
class_index:688, Acc1 0.8715384615384615
1005
Len of new dataset is :1005
class_index:689, Acc1 0.47562189054726367
1300
Len of new dataset is :1300
class_index:690, Acc1 0.78
1300
Len of new dataset is :1300
class_index:691, Acc1 0.7176923076923077
1300
Len of new dataset is :1300
class_index:692, Acc1 0.6069230769230769
1300

/data1/jiaxin/anaconda3/envs/Py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


class_index:782, Acc1 0.39384372533543804
1300
Len of new dataset is :1300
class_index:783, Acc1 0.6084615384615385
1300
Len of new dataset is :1300
class_index:784, Acc1 0.4461538461538462
1300
Len of new dataset is :1300
class_index:785, Acc1 0.7284615384615385
1300
Len of new dataset is :1300
class_index:786, Acc1 0.803076923076923
1300
Len of new dataset is :1300
class_index:787, Acc1 0.5376923076923077
1300
Len of new dataset is :1300
class_index:788, Acc1 0.5984615384615385
1249
Len of new dataset is :1249
class_index:789, Acc1 0.7421937550040032
1300
Len of new dataset is :1300
class_index:790, Acc1 0.7776923076923077
1300
Len of new dataset is :1300
class_index:791, Acc1 0.75
1300
Len of new dataset is :1300
class_index:792, Acc1 0.48615384615384616
1300
Len of new dataset is :1300
class_index:793, Acc1 0.7861538461538462
1300
Len of new dataset is :1300
class_index:794, Acc1 0.7769230769230769
1300
Len of new dataset is :1300
class_index:795, Acc1 0.7
1300
Len of new dataset i

1300
Len of new dataset is :1300
class_index:895, Acc1 0.7384615384615385
1300
Len of new dataset is :1300
class_index:896, Acc1 0.666923076923077
1300
Len of new dataset is :1300
class_index:897, Acc1 0.8353846153846154
1300
Len of new dataset is :1300
class_index:898, Acc1 0.4946153846153846
1300
Len of new dataset is :1300
class_index:899, Acc1 0.30846153846153845
1300
Len of new dataset is :1300
class_index:900, Acc1 0.8938461538461538
1055
Len of new dataset is :1055
class_index:901, Acc1 0.7156398104265402
1300
Len of new dataset is :1300
class_index:902, Acc1 0.5492307692307692
1300
Len of new dataset is :1300
class_index:903, Acc1 0.6892307692307692
1300
Len of new dataset is :1300
class_index:904, Acc1 0.6876923076923077
1300
Len of new dataset is :1300
class_index:905, Acc1 0.703076923076923
1213
Len of new dataset is :1213
class_index:906, Acc1 0.5869744435284419
1300
Len of new dataset is :1300
class_index:907, Acc1 0.56
1300
Len of new dataset is :1300
class_index:908, Acc

In [13]:
rbg_gt_pred_acc = "./rbg_gt_pred_acc.pickle"
with open(rbg_gt_pred_acc, "wb") as handle:
    pickle.dump(rbg_class_y_true_y_pred_acc, handle)
print(np.mean([acc for _, _, acc in rbg_class_y_true_y_pred_acc.values()]))

0.6091651732689805


## Load Swap, RGB->BRG

In [14]:
brg_class_y_true_y_pred_acc = {}
for class_index in range(1000):
    # Load data
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root=root, 
                                 is_train=True, transform=train_transform)
    len_specific_category = len(trainset.targets_imgs_dict[class_index])
    print(len_specific_category)
    trainset.set_data([class_index], len_specific_category)
    trainloader = torch.utils.data.DataLoader(trainset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=num_workers,
                                              pin_memory=True)
    y_true, y_pred, acc = get_gt_pred_specific_class(net, 
                                                     trainloader,
                                                     order=[2,0,1],
                                                     device=device)
    print("class_index:{}, Acc1 {}".format(class_index, acc))
    brg_class_y_true_y_pred_acc[class_index] = [y_true, y_pred, acc]

1300
Len of new dataset is :1300
class_index:0, Acc1 0.3353846153846154
1300
Len of new dataset is :1300
class_index:1, Acc1 0.25
1300
Len of new dataset is :1300
class_index:2, Acc1 0.42615384615384616
1300
Len of new dataset is :1300
class_index:3, Acc1 0.5615384615384615
1300
Len of new dataset is :1300
class_index:4, Acc1 0.6292307692307693
1300
Len of new dataset is :1300
class_index:5, Acc1 0.46692307692307694
1300
Len of new dataset is :1300
class_index:6, Acc1 0.4976923076923077
1300
Len of new dataset is :1300
class_index:7, Acc1 0.3484615384615385
1300
Len of new dataset is :1300
class_index:8, Acc1 0.21153846153846154
1300
Len of new dataset is :1300
class_index:9, Acc1 0.8292307692307692
1300
Len of new dataset is :1300
class_index:10, Acc1 0.1823076923076923
1300
Len of new dataset is :1300
class_index:11, Acc1 0.19692307692307692
1300
Len of new dataset is :1300
class_index:12, Acc1 0.25
1300
Len of new dataset is :1300
class_index:13, Acc1 0.68
1300
Len of new dataset is

1300
Len of new dataset is :1300
class_index:115, Acc1 0.5769230769230769
1300
Len of new dataset is :1300
class_index:116, Acc1 0.6446153846153846
1300
Len of new dataset is :1300
class_index:117, Acc1 0.7346153846153847
1300
Len of new dataset is :1300
class_index:118, Acc1 0.05923076923076923
1300
Len of new dataset is :1300
class_index:119, Acc1 0.12923076923076923
1300
Len of new dataset is :1300
class_index:120, Acc1 0.4107692307692308
1300
Len of new dataset is :1300
class_index:121, Acc1 0.03461538461538462
1300
Len of new dataset is :1300
class_index:122, Acc1 0.10307692307692308
1300
Len of new dataset is :1300
class_index:123, Acc1 0.5884615384615385
1300
Len of new dataset is :1300
class_index:124, Acc1 0.15
1300
Len of new dataset is :1300
class_index:125, Acc1 0.3123076923076923
1300
Len of new dataset is :1300
class_index:126, Acc1 0.54
1300
Len of new dataset is :1300
class_index:127, Acc1 0.4469230769230769
1300
Len of new dataset is :1300
class_index:128, Acc1 0.54615

class_index:226, Acc1 0.3638461538461538
1300
Len of new dataset is :1300
class_index:227, Acc1 0.5153846153846153
1300
Len of new dataset is :1300
class_index:228, Acc1 0.8092307692307692
1300
Len of new dataset is :1300
class_index:229, Acc1 0.6607692307692308
1300
Len of new dataset is :1300
class_index:230, Acc1 0.3953846153846154
1300
Len of new dataset is :1300
class_index:231, Acc1 0.2523076923076923
1300
Len of new dataset is :1300
class_index:232, Acc1 0.5707692307692308
1300
Len of new dataset is :1300
class_index:233, Acc1 0.4707692307692308
1300
Len of new dataset is :1300
class_index:234, Acc1 0.018461538461538463
1300
Len of new dataset is :1300
class_index:235, Acc1 0.11307692307692307
1300
Len of new dataset is :1300
class_index:236, Acc1 0.19538461538461538
1300
Len of new dataset is :1300
class_index:237, Acc1 0.3607692307692308
1300
Len of new dataset is :1300
class_index:238, Acc1 0.17461538461538462
1300
Len of new dataset is :1300
class_index:239, Acc1 0.134615384

class_index:338, Acc1 0.6530769230769231
1300
Len of new dataset is :1300
class_index:339, Acc1 0.3269230769230769
1300
Len of new dataset is :1300
class_index:340, Acc1 0.9261538461538461
1300
Len of new dataset is :1300
class_index:341, Acc1 0.37538461538461537
1300
Len of new dataset is :1300
class_index:342, Acc1 0.4453846153846154
1300
Len of new dataset is :1300
class_index:343, Acc1 0.6015384615384616
1300
Len of new dataset is :1300
class_index:344, Acc1 0.5938461538461538
1300
Len of new dataset is :1300
class_index:345, Acc1 0.45153846153846156
1300
Len of new dataset is :1300
class_index:346, Acc1 0.49846153846153846
1300
Len of new dataset is :1300
class_index:347, Acc1 0.6184615384615385
1300
Len of new dataset is :1300
class_index:348, Acc1 0.4646153846153846
1300
Len of new dataset is :1300
class_index:349, Acc1 0.6276923076923077
1300
Len of new dataset is :1300
class_index:350, Acc1 0.6376923076923077
1300
Len of new dataset is :1300
class_index:351, Acc1 0.46538461538

class_index:450, Acc1 0.7184615384615385
1300
Len of new dataset is :1300
class_index:451, Acc1 0.7269230769230769
1300
Len of new dataset is :1300
class_index:452, Acc1 0.6892307692307692
1300
Len of new dataset is :1300
class_index:453, Acc1 0.6761538461538461
1300
Len of new dataset is :1300
class_index:454, Acc1 0.5661538461538461
1300
Len of new dataset is :1300
class_index:455, Acc1 0.7384615384615385
1300
Len of new dataset is :1300
class_index:456, Acc1 0.5223076923076924
1300
Len of new dataset is :1300
class_index:457, Acc1 0.4276923076923077
1300
Len of new dataset is :1300
class_index:458, Acc1 0.7538461538461538
1300
Len of new dataset is :1300
class_index:459, Acc1 0.49846153846153846
1300
Len of new dataset is :1300
class_index:460, Acc1 0.37384615384615383
1300
Len of new dataset is :1300
class_index:461, Acc1 0.3830769230769231
1300
Len of new dataset is :1300
class_index:462, Acc1 0.6876923076923077
1300
Len of new dataset is :1300
class_index:463, Acc1 0.583076923076

1300
Len of new dataset is :1300
class_index:562, Acc1 0.7769230769230769
1300
Len of new dataset is :1300
class_index:563, Acc1 0.5392307692307692
1300
Len of new dataset is :1300
class_index:564, Acc1 0.75
1300
Len of new dataset is :1300
class_index:565, Acc1 0.8676923076923077
1300
Len of new dataset is :1300
class_index:566, Acc1 0.666923076923077
1222
Len of new dataset is :1222
class_index:567, Acc1 0.26595744680851063
1300
Len of new dataset is :1300
class_index:568, Acc1 0.37461538461538463
1300
Len of new dataset is :1300
class_index:569, Acc1 0.8430769230769231
1300
Len of new dataset is :1300
class_index:570, Acc1 0.6907692307692308
1300
Len of new dataset is :1300
class_index:571, Acc1 0.7107692307692308
1300
Len of new dataset is :1300
class_index:572, Acc1 0.7715384615384615
1300
Len of new dataset is :1300
class_index:573, Acc1 0.7369230769230769
1300
Len of new dataset is :1300
class_index:574, Acc1 0.7923076923076923
1300
Len of new dataset is :1300
class_index:575, A

1159
Len of new dataset is :1159
class_index:675, Acc1 0.6125970664365833
1133
Len of new dataset is :1133
class_index:676, Acc1 0.5489849955869374
1300
Len of new dataset is :1300
class_index:677, Acc1 0.5346153846153846
1180
Len of new dataset is :1180
class_index:678, Acc1 0.40508474576271186
1300
Len of new dataset is :1300
class_index:679, Acc1 0.8084615384615385
1300
Len of new dataset is :1300
class_index:680, Acc1 0.5915384615384616
1300
Len of new dataset is :1300
class_index:681, Acc1 0.42153846153846153
1300
Len of new dataset is :1300
class_index:682, Acc1 0.8538461538461538
1300
Len of new dataset is :1300
class_index:683, Acc1 0.4469230769230769
1300
Len of new dataset is :1300
class_index:684, Acc1 0.676923076923077
1300
Len of new dataset is :1300
class_index:685, Acc1 0.9176923076923077
1120
Len of new dataset is :1120
class_index:686, Acc1 0.6303571428571428
1300
Len of new dataset is :1300
class_index:687, Acc1 0.7307692307692307
1300
Len of new dataset is :1300
clas

/data1/jiaxin/anaconda3/envs/Py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


class_index:782, Acc1 0.39463299131807417
1300
Len of new dataset is :1300
class_index:783, Acc1 0.6123076923076923
1300
Len of new dataset is :1300
class_index:784, Acc1 0.41307692307692306
1300
Len of new dataset is :1300
class_index:785, Acc1 0.6069230769230769
1300
Len of new dataset is :1300
class_index:786, Acc1 0.7853846153846153
1300
Len of new dataset is :1300
class_index:787, Acc1 0.5053846153846154
1300
Len of new dataset is :1300
class_index:788, Acc1 0.6246153846153846
1249
Len of new dataset is :1249
class_index:789, Acc1 0.5788630904723779
1300
Len of new dataset is :1300
class_index:790, Acc1 0.7638461538461538
1300
Len of new dataset is :1300
class_index:791, Acc1 0.73
1300
Len of new dataset is :1300
class_index:792, Acc1 0.4938461538461538
1300
Len of new dataset is :1300
class_index:793, Acc1 0.6807692307692308
1300
Len of new dataset is :1300
class_index:794, Acc1 0.7846153846153846
1300
Len of new dataset is :1300
class_index:795, Acc1 0.69
1300
Len of new dataset

1300
Len of new dataset is :1300
class_index:895, Acc1 0.6738461538461539
1300
Len of new dataset is :1300
class_index:896, Acc1 0.6553846153846153
1300
Len of new dataset is :1300
class_index:897, Acc1 0.8476923076923077
1300
Len of new dataset is :1300
class_index:898, Acc1 0.42615384615384616
1300
Len of new dataset is :1300
class_index:899, Acc1 0.28692307692307695
1300
Len of new dataset is :1300
class_index:900, Acc1 0.8930769230769231
1055
Len of new dataset is :1055
class_index:901, Acc1 0.6919431279620853
1300
Len of new dataset is :1300
class_index:902, Acc1 0.5161538461538462
1300
Len of new dataset is :1300
class_index:903, Acc1 0.6553846153846153
1300
Len of new dataset is :1300
class_index:904, Acc1 0.6461538461538462
1300
Len of new dataset is :1300
class_index:905, Acc1 0.67
1213
Len of new dataset is :1213
class_index:906, Acc1 0.49134377576257215
1300
Len of new dataset is :1300
class_index:907, Acc1 0.5838461538461538
1300
Len of new dataset is :1300
class_index:908,

In [15]:
brg_gt_pred_acc = "./brg_gt_pred_acc.pickle"
with open(brg_gt_pred_acc, "wb") as handle:
    pickle.dump(brg_class_y_true_y_pred_acc, handle)
print(np.mean([acc for _, _, acc in brg_class_y_true_y_pred_acc.values()]))

0.5428853405058999
